<h1 style='color:red', align='center'>Model creation </h1>

## Importing libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os
import json

## Creation of Image size, Batch size, Epochs

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 12

## Observation
Creation of **Image size**,**Batch size**,**Epochs** are successfully created.

In [3]:
train_dir = "data/train"
val_dir = "data/validation"

## Data Generation for Training and Testing

In [4]:
# Data Generation
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

In [5]:
val_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

## Observation
The training and testing are successfully created.

In [6]:
train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 836 images belonging to 20 classes.


In [7]:
val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 743 images belonging to 20 classes.


## Base model

In [8]:
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

print("Base model output shape:", base_model.output_shape)


Base model output shape: (None, 7, 7, 1280)


## Build model

In [9]:
# Build model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1280)                │           5,120 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │           5,140 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,596,180 (9.90 MB)

 Trainable params: 335,636 (1.28 MB)

 Non-trainable params: 2,260,544 (8.62 MB)

## Observation
Model is created successfully.

## Accuracy

In [10]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS
)

Epoch 1/12
27/27 ━━━━━━━━━━━━━━━━━━━━ 129s 4s/step - accuracy: 0.0837 - loss: 4.0101 - val_accuracy: 0.1669 - val_loss: 2.6964
Epoch 2/12
27/27 ━━━━━━━━━━━━━━━━━━━━ 130s 4s/step - accuracy: 0.1998 - loss: 2.9554 - val_accuracy: 0.3311 - val_loss: 2.2136
Epoch 3/12
27/27 ━━━━━━━━━━━━━━━━━━━━ 105s 4s/step - accuracy: 0.2919 - loss: 2.4600 - val_accuracy: 0.4536 - val_loss: 1.9269
Epoch 4/12
27/27 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.3624 - loss: 2.1361 - val_accuracy: 0.5666 - val_loss: 1.6511
Epoch 5/12
27/27 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 0.3852 - loss: 2.0078 - val_accuracy: 0.6231 - val_loss: 1.4329
Epoch 6/12
27/27 ━━━━━━━━━━━━━━━━━━━━ 151s 4s/step - accuracy: 0.4665 - loss: 1.7409 - val_accuracy: 0.6972 - val_loss: 1.2423
Epoch 7/12
27/27 ━━━━━━━━━━━━━━━━━━━━ 105s 4s/step - accuracy: 0.5191 - loss: 1.5231 - val_accuracy: 0.7510 - val_loss: 1.0861
Epoch 8/12
27/27 ━━━━━━━━━━━━━━━━━━━━ 107s 4s/step - accuracy: 0.5239 - loss: 1.5733 - val_accuracy: 0.7900 - v

In [11]:
# save model
model.save("cricketer_model.h5")

## Observation 
The model saved under the **cricketer_model.keras** is done successfully.

## Creation of Json file

In [12]:
with open("class_indices.json","w") as f:
    json.dump(train_data.class_indices,f)

## Observation
- Creation of Json file is successfully created and it is named as **class_indices**.
- It is Cricketer name classes.

In [13]:
print("✅ Training complete and model saved")

✅ Training complete and model saved
